In [1]:
import keras
import keras.backend as K
from keras import initializers
import h5py
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

In [35]:
import numpy as np
import random

class G():
  
  def __init__(self, model_fixed, clean_dataset,
               num_overlay=6,
               clean_data_overlay_rate=0.4,
               input_data_overlay_rate=0.9,
               cls=1283,
               threshold = 0.01):
    # super(G, self).__init__()
    self.model_fixed = model_fixed
    self.clean_dataset = clean_dataset
    self.num_overlay = num_overlay
    self.clean_data_overlay_rate = clean_data_overlay_rate
    self.input_data_overlay_rate = input_data_overlay_rate
    self.cls = 1283
    self.threshold = threshold
  
  def overlay(self, x, clean_img):
    overlay_img = x * self.input_data_overlay_rate + clean_img * self.clean_data_overlay_rate
    return np.clip(overlay_img, 0.0, 1.0)
  
  def predict(self, x):
    final_out = []
    for i in range(x.shape[0]):
      img_shape = x[i].shape
      overlay_res = np.zeros(self.num_overlay)
      for j in range(self.num_overlay):
        overlay_img = self.overlay(x[i],
                                   self.clean_dataset[random.randint(0, self.clean_dataset.shape[0]-1)])
        overlay_img = np.asarray(overlay_img).reshape((1,
                                                       img_shape[0],
                                                       img_shape[1],
                                                       img_shape[2]))
        overlay_res[j] = np.argmax(self.model_fixed.predict(overlay_img))
      if np.var(overlay_res) < self.threshold:
        # smaller variance on overlayed prediction => poisoned
        output_G = np.zeros(self.cls + 1)
        output_G[self.cls] = 1
      else:
        # bigger variance on overlayed prediction => not poisoned
        x_single = np.asarray(x[i]).reshape((1,
                                             img_shape[0],
                                             img_shape[1],
                                             img_shape[2]))
        output_G = np.append(self.model_fixed.predict(x_single), 0)
      final_out.append(output_G)
    return np.array(final_out)

In [20]:
# supposely "eval.py" for alternative G class


clean_data_filename = '/content/gdrive/MyDrive/ml_cyber_data/clean_test_data.h5'
clean_data_val_filename = '/content/gdrive/MyDrive/ml_cyber_data/clean_validation_data.h5'
poisoned_data_filename = '/content/gdrive/MyDrive/ml_cyber_data/sunglasses_poisoned_data.h5'
repaired_model_filename = '/content/gdrive/MyDrive/ml_cyber_data/sunglasses_bd_net.h5'
# repaired_model_filename = '/content/gdrive/MyDrive/ml_cyber_data/sunglasses_fine_pruning_model.h5'



cl_x_test, cl_y_test = data_loader(clean_data_filename)
bd_x_test, bd_y_test = data_loader(poisoned_data_filename)
clean_val, clean_val_y = data_loader(clean_data_val_filename)

n = 100
cl_x_test = cl_x_test[:n]
cl_y_test = cl_y_test[:n]
bd_x_test = bd_x_test[:n]
bd_y_test = bd_y_test[:n]
clean_val = clean_val[:n]


repaired_model = keras.models.load_model(repaired_model_filename, compile=False)



In [36]:
goodnet = G(repaired_model, clean_val/255)

cl_label_p = np.argmax(goodnet.predict(cl_x_test/255), axis=1)
clean_accuracy = np.mean(np.equal(cl_label_p, cl_y_test)) * 100
print('Clean Classification accuracy:', clean_accuracy)

bd_label_p = np.argmax(goodnet.predict(bd_x_test/255), axis=1)
asr = np.mean(np.equal(bd_label_p, bd_y_test)) * 100
print('Attack Success Rate:', asr)

Clean Classification accuracy: 87.0
Attack Success Rate: 4.0
